In [ ]:
from io import StringIO
from itertools import chain

import httpx
import pandas as pd


In [ ]:
res = httpx.get(
    "https://digital.feprecisionplus.com/greateasternlife/en-GB/GreatEastern_V2/DownloadTool/FundOptions",
    params={
        "GrsProjectId": 95400076,
        "ProjectName": "greateasternlife",
        "ToolId": 16,
        "LanguageId": 1,
        "LanguageCode": "en-GB",
    },
)

In [ ]:
all_funds = pd.read_xml(StringIO(res.json()["Html"]), xpath="//option").dropna()

In [ ]:
greatlink_filter = httpx.get(
    "https://digitalfundservice.feprecisionplus.com/FundDataService.svc/GetRowIdList",
    params={
        "jsonString": '{"FilteringOptions":{"undefined":0,"CategoryId":"GRE","RangeId":null,"RangeName":"","Category2Id":null,"PriipProductCode":null,"DefaultCategoryId":null,"DefaultCategory2Id":null,"ForSaleIn":null,"ShowMainUnits":false,"MPCategoryCode":null},"ProjectName":"greateasternlife","LanguageCode":"en-GB","UserType":"","Region":"","LanguageId":"1","Theme":"GreatEastern_V2","SortingStyle":"1","PageNo":1,"PageSize":10,"OrderBy":"UnitName:init","IsAscOrder":true,"OverrideDocumentCountryCode":null,"ToolId":"1","PrefetchPages":200,"PrefetchPageStart":1,"OverridenThemeName":"GreatEastern_V2","ForSaleIn":"","ValidateFeResearchAccess":false,"HasFeResearchFullAccess":false,"EnableSedolSearch":"false","GrsProjectId":"95400076","ShowMainUnitExpansion":false,"UseCombinedOngoingChargeTER":false}'
    },
)

In [ ]:
greatlink_rows = list(
    chain.from_iterable(
        [map(int, row.split(",")) for row in greatlink_filter.json()["RowIdList"]]
    )
)

In [ ]:
greatlink_with_dividend_filter = httpx.get(
    "https://digitalfundservice.feprecisionplus.com/FundDataService.svc/GetRowIdList",
    params={
        "jsonString": '{"FilteringOptions":{"undefined":0,"CategoryId":"GRE","BespokeColumn5":"Yes - Payout (with option to reinvest)","RangeId":null,"RangeName":"","Category2Id":null,"PriipProductCode":null,"DefaultCategoryId":null,"DefaultCategory2Id":null,"ForSaleIn":null,"ShowMainUnits":false,"MPCategoryCode":null},"ProjectName":"greateasternlife","LanguageCode":"en-GB","UserType":"","Region":"","LanguageId":"1","Theme":"GreatEastern_V2","SortingStyle":"1","PageNo":1,"PageSize":10,"OrderBy":"UnitName:init","IsAscOrder":true,"OverrideDocumentCountryCode":null,"ToolId":"1","PrefetchPages":200,"PrefetchPageStart":1,"OverridenThemeName":"GreatEastern_V2","ForSaleIn":"","ValidateFeResearchAccess":false,"HasFeResearchFullAccess":false,"EnableSedolSearch":"false","GrsProjectId":"95400076","ShowMainUnitExpansion":false,"UseCombinedOngoingChargeTER":false}'
    },
)

In [ ]:
greatlink_with_dividend_rows = list(
    chain.from_iterable(
        [
            map(int, row.split(","))
            for row in greatlink_with_dividend_filter.json()["RowIdList"]
        ]
    )
)

In [ ]:
for _, row in all_funds.loc[greatlink_rows].iterrows():
    res = httpx.get(
        "https://digital.feprecisionplus.com/greateasternlife/en-GB/GreatEastern_V2/DownloadTool/GetPriceHistoryFile",
        params={
            "modelString": {
                "GrsProjectId": "95400076",
                "ProjectName": "greateasternlife",
                "ToolId": 16,
                "LanguageId": "1",
                "LanguageCode": "en-GB",
                "forSaleIn": "",
                "FSIexclCT": "",
            },
            "filtersString": {
                "CitiCode": row["value"],
                "Universe": "SI",
                "TypeCode": row["typecode"],
                "FundName": row["option"],
                "BaseCurrency": "SGD",
                "PriceType": 2,
                "TimePeriod": "",
            },
        },
        timeout=30,
    )
    if res.is_success:
        with open(
            f"data/GreatLink/{res.headers['Content-Disposition'].split('filename=')[1].split('"')[1].split('_')[0]}.xlsx",
            "wb",
        ) as f:
            f.write(res.content)
    else:
        print(
            f"Failed to download {row['option']} {row['typecode']}: {res.status_code} {res.text}"
        )

In [ ]:
for _, row in all_funds.loc[greatlink_with_dividend_rows].iterrows():
    res = httpx.get(
        "https://digital.feprecisionplus.com/greateasternlife/en-GB/GreatEastern_V2/DownloadTool/GetDividendHistoryFile",
        params={
            "modelString": {
                "GrsProjectId": "95400076",
                "ProjectName": "greateasternlife",
                "ToolId": 16,
                "LanguageId": "1",
                "LanguageCode": "en-GB",
                "forSaleIn": "",
                "FSIexclCT": "",
            },
            "filtersString": {
                "TypeCode": row["typecode"],
                "FundName": row["option"],
                "BaseCurrency": "SGD",
                "PriceType": 2,
                "TimePeriod": "",
            },
        },
        timeout=30,
    )
    if res.is_success:
        with open(
            f"data/GreatLink/{res.headers['Content-Disposition'].split('filename=')[1].split('"')[1].split('_')[0]}_Dividends.xlsx",
            "wb",
        ) as f:
            f.write(res.content)
    else:
        print(
            f"Failed to download {row['option']} {row['typecode']}: {res.status_code} {res.text}"
        )